<a href="https://colab.research.google.com/github/sheon-j/webcrawling-study/blob/main/sojoong_crawler_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install selenium==3.14.0

In [ ]:
# import requests
# import re
# from bs4 import BeautifulSoup
# import selenium
# # pip install selenium==3.14.0
# from selenium import webdriver
# from selenium.webdriver import ActionChains
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import time

# ## selenium 사용
# options = webdriver.ChromeOptions()
# # 창이 뜨지 않게 설정
# # options.add_argument('headless')
# options.headless = False
# # 창 크기 조절 설정
# options.add_argument('window-size=1920x1080')
# # 그래픽 카드 기능 제거
# options.add_argument("disable-gpu")
# # F11 전체 화면 설정
# # options.add_argument('--start-fullscreen')
# # user-agent 값 입력
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36")
# # 해당 국가 언어값 입력
# options.add_argument("lang=ko_KR")

# # directory = r'C:\Users\<username>\Desktop\Python\CrawlingTest'

# # options.add_experimental_option("prefs", {
# #   "download.default_directory": directory,
# #   "download.prompt_for_download": False,
# #   "download.directory_upgrade": True,
# #   "safebrowsing.enabled": True
# # })

# # Driver & Web Load
# url = "https://sojoong.joins.com/archives/category/news"
# driver = webdriver.Chrome(executable_path=r"C:\Util\chromedriver_win32\chromedriver.exe", chrome_options=options)

# # driver = webdriver.Chrome(r"C:\Util\chromedriver_win32\chromedriver.exe")

# driver.get(url)
# # driver.maximize_window()
# for i in range(2):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     driver.find_element_by_xpath("//tr[@class='btn_more']").click()
#     time.sleep(1)

# # print(html)

# html = driver.page_source
# soup = BeautifulSoup(html, "html.parser")
# items = soup.select('#list_mypost > table > tbody > tr')
# for item in items:
#     a_tag = item.select_one('td.title > h2 > a')
#     if a_tag is not None:
#         print(a_tag.text)

## full web crawling
* BeautifulSoup
* Crawling Target
  1. url
  2. title
  3. author
  4. date
  5. article
  6. img_path
  7. source

In [542]:
# 소년중앙 크롤링

%%time
import requests
import re
from bs4 import BeautifulSoup

url = "https://sojoong.joins.com/archives/category/nie/issue"
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, "html.parser")

news_list = soup.select("h2 > a")
news_list = [p.get("href") for p in news_list]

data = []
for news in news_list:
    json_data = {}
    # sub crawling
    sub_req = requests.get(news)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, "html.parser")

    # url: 기사 원문 URL
    json_data["url"] = news

    # title: 제목
    title = sub_soup.select_one("h1").text
    title = re.sub("[\r\n]", "", title).strip()
    json_data["title"] = title

    # author: 기자
    # date: 날짜
    meta = sub_soup.select_one("div.post-meta").text.split("|")
    author = meta[0].strip()
    date = meta[1].strip()

    json_data["author"] = author
    json_data["date"] = date

    # article: 기사 내용
    article = []
    p_list = sub_soup.select("p")
    end_point = None
    
    for i, p in enumerate(p_list):
        if p.get("class"):
            break
        text = re.sub("[\xa0\u200b\n\r\t]", "",p.text).strip()
        if text[-2:] == "일자":
            end_point = i + 1
        article.append(text)
    
    article = " ".join(article[:end_point])
    json_data["article"] = article

    # img_path: 기사 img 경로
    img_path = sub_soup.select_one("figure").a.get("href")
    json_data["img_path"] = img_path

    # source: 신문사
    json_data["source"] = "소년중앙"

    data.append(json_data)

CPU times: user 1.03 s, sys: 22.9 ms, total: 1.05 s
Wall time: 12.2 s


In [553]:
for key in data[0]:
    print(f"{key}: {data[0][key][:100]}")

url: https://sojoong.joins.com/archives/44661
title: [뉴스레시피] 화산 폭발로 큰 피해를 입은  통가를 돕기 위해 힘을 모아봐요
author: 한은정 기자
date: 2022.01.24 08:02
article: 대폭발로 사라진 화산섬···통가의 영웅 “조국 도와달라” “통가는 도움이 절실합니다.” 남태평양 섬나라 통가의 태권도 국가대표 선수인 피타 타우파토푸아(38)가 화산 폭발과 지진해
img_path: https://sojoong.joins.com/wp-content/uploads/sites/4/2022/01/IM00007568_20220119-143142.jpg
source: 소년중앙
